<a href="https://colab.research.google.com/github/rajagopalmotivate1/AILab/blob/master/4_Medical_image_classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from urllib.request import urlopen

urls = {'pneumothorax_test':'https://www.dropbox.com/s/x74ykyivipwnozs/pneumothorax_test.h5?dl=1',
        'pneumothorax_train':'https://www.dropbox.com/s/pnwf67qzztd1slc/pneumothorax_train.h5?dl=1'}

for (name,url) in urls.items():
        print('Downloading '+name+'...')
        u = urlopen(url)
        data = u.read()
        u.close()

        with open(name+'.h5', "wb") as f :
            f.write(data)
print('Files have been downloaded.')

In [0]:
import h5py as h5

X_trainDownloaded = h5.File('pneumothorax_train.h5')['image'][:]
Y_trainDownloaded = h5.File('pneumothorax_train.h5')['label'][:]

X_test = h5.File('pneumothorax_test.h5')['image'][:]
Y_test = h5.File('pneumothorax_test.h5')['label'][:]

#Set the no of examples  

In [0]:
print('no of examples in the downloaed dataset = ' + str(X_trainDownloaded.shape[0]))

In [0]:
noofexamples = 100

In [0]:
X_train = X_trainDownloaded[0:noofexamples]
Y_train = Y_trainDownloaded[0:noofexamples]

In [0]:
print('no of examples = ' + str(X_train.shape[0]))

In [0]:

num_train=X_train.shape[0]
num_test=X_test.shape[0]
X_train.shape

In [0]:
from keras.models import Model,Sequential
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.layers.convolutional import Conv2D,SeparableConv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten,GlobalAveragePooling2D
from keras.layers import Dense,Input
from keras.layers import Activation
from keras.layers import Dropout
from keras.optimizers import SGD,Adadelta
from keras.optimizers import Adagrad
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import tensorflow as tf

In [0]:
seed=1337
np.random.seed(seed)

In [0]:
X_train = (X_train/255).astype('float32')
X_test = (X_test/255).astype('float32')
Y_train = Y_train.astype('float32')
Y_test = Y_test.astype('float32')
num_class=2
X_train.shape[1:]

In [0]:
inputs=Input(shape=(256,256,1))
  
x=Conv2D(32, (5,5), padding='same',kernel_initializer='random_uniform', activation='relu')(inputs)
y=Conv2D(32, (5,5), padding='same', kernel_initializer='random_uniform',activation='relu')(x)
z=MaxPooling2D(pool_size=(2,2))(keras.layers.add([x,y]))
  
x=Conv2D(64, (5,5), padding='same',kernel_initializer='random_uniform', activation='relu')(z)
y=Conv2D(64, (5,5), padding='same',kernel_initializer='random_uniform', activation='relu')(x)
z=MaxPooling2D(pool_size=(2,2))(keras.layers.add([x,y]))
  
x=SeparableConv2D(128, (5,5), padding='same',kernel_initializer='random_uniform', activation='relu')(z)
y=SeparableConv2D(128, (5,5), padding='same',kernel_initializer='random_uniform', activation='relu')(x)
z=MaxPooling2D(pool_size=(2,2))(keras.layers.add([x,y]))
  
x=Conv2D(256, (3,3), padding='same',kernel_initializer='random_uniform', activation='relu')(z)
y=Conv2D(256, (3,3), padding='same',kernel_initializer='random_uniform', activation='relu')(x)
y=Conv2D(256, (3,3), padding='same',kernel_initializer='random_uniform', activation='relu')(y)
z=MaxPooling2D(pool_size=(2,2))(keras.layers.add([x,y]))

z=GlobalAveragePooling2D()(z)

x=Dense(64,activation='relu',)(z)
x=Dense(2,activation='sigmoid',kernel_initializer='random_uniform')(x)

model=Model(inputs = inputs, outputs = x)
#step 2: Learning target(computiong the loss using the entropy function
sgd = SGD(lr=0.06, decay=1e-6, momentum=0.9, nesterov=True)

adagrad = Adagrad(lr=0.05, epsilon=1e-08)
opt = Adadelta(lr=1.0, rho=0.95, epsilon=0.0001, decay=0.000001)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [0]:

print ('Training Start....')
#reset_weights(model)
training = model.fit(X_train,Y_train, batch_size=90,epochs=20,verbose=1, validation_data=(X_test, Y_test),shuffle=False)

In [0]:
score = model.evaluate(X_test,Y_test,verbose=1)
print("CNN Error: %.2f%%" % (100-score[1]*100))
'CNN Accuracy: %.2f%%' % (score[1]*100)

In [0]:
import matplotlib.pyplot as plt

plt.plot(training.history['acc'])

plt.plot(training.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training accuracy', 'Validation accuracy'], loc='upper left')

plt.show()

In [0]:
import matplotlib.pyplot as plt

plt.plot(training.history['loss'])

plt.plot(training.history['val_loss'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training loss', 'Validation loss'], loc='upper left')

plt.show()

In [0]:
print( 'Predicted class is ' + str(np.argmax(Y_test[1200])))
test=X_test[1200]
test=test.reshape(1,256,256,1)

print('Ground truth = ')
print(Y_test[1200])

print('Predicted class = ')
model.predict(test)

